## Map deCODE genetic map coordinates

Downloaded [sex averaged recombination rates](https://www.decode.com/additional/sex-averaged_noncarrier.rmap). The genome coordinates are for hg18, according to the [original Nature publication](http://www.nature.com/nature/journal/v467/n7319/full/nature09525.html). This corresponds to [NCBI Build 36.1](https://genome.ucsc.edu/FAQ/FAQreleases.html). This notebook maps coordinates onto the latest release (Genome Reference Consortium GRCh38).

The contents of the recombination rate file are [described here](https://www.decode.com/addendum/).

The coordinate mapping is done using a python implementation of [UCSC LiftOver](https://genome.ucsc.edu/cgi-bin/hgLiftOver). A tab delimited file with the original and new coordinate is written.

In [ ]:
from pyliftover import LiftOver
import pandas as pd
from cogent3 import LoadTable
import os

def CoordMapper(lo):
    def process_row(chrom_pos):
        chrom, pos = chrom_pos
        new = lo.convert_coordinate(chrom, int(pos))
        if not new or len(new) > 1:
            return None

        new = new[0]
        if chrom != new[0]:
            # different chrom
            return None
        
        new_pos = new[1]
        
        return new_pos
    return process_row

Download deCODE recombination maps, then convert to `cogent3.Table`.

In [ ]:
path = "..."     #path to save resulting file
if not os.getcwd() == path:
    os.chdir(path)
!curl -O https://www.decode.com/additional/sex-averaged_noncarrier.rmap
filename = "sex-averaged_noncarrier.rmap"
df = pd.read_csv(filename, sep='\t')
table = LoadTable(data_frame=df)

Use pyliftover to convert to hg38 coordinates

In [ ]:
lo = LiftOver("hg18", "hg38")

mapper = CoordMapper(lo)

new = table.with_new_column("pos38", mapper, columns=["chr", "pos"])
# exclude positions without a coordinate
new = new.filtered(lambda x: x is not None, columns="pos38")
new = new.sorted(columns=["chr", "pos38"])
new.write("sex-averaged_noncarrier-hg38.csv", sep=',')
new[:20]